## Simple GAN

### Defining Discriminator

In [17]:
from keras import Sequential
from tensorflow.keras.layers import Conv2D, LeakyReLU, Dropout, Flatten, Dense, Activation, BatchNormalization, Reshape, UpSampling2D, Conv2DTranspose
from keras.optimizers import RMSprop
import numpy as np

In [21]:
class GAN():
  def __init__(self, batch_size):
    self.D = Sequential()
    depth = 64
    dropout = 0.4
    # In: 28 x 28 x 1, depth = 1
    # Out: 14 x 14 x 1, depth=64
    input_shape = (self.img_rows, self.img_cols, self.channel)
    self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
    padding='same', activation=LeakyReLU(alpha=0.2)))
    self.D.add(Dropout(dropout))
    self.D.add(Conv2D(depth*2, 5, strides=2, padding='same',\
    activation=LeakyReLU(alpha=0.2)))
    self.D.add(Dropout(dropout))
    self.D.add(Conv2D(depth*4, 5, strides=2, padding='same',\
    activation=LeakyReLU(alpha=0.2)))
    self.D.add(Dropout(dropout))
    self.D.add(Conv2D(depth*8, 5, strides=1, padding='same',\
    activation=LeakyReLU(alpha=0.2)))
    self.D.add(Dropout(dropout))
    # Out: 1-dim probability
    self.D.add(Flatten())
    self.D.add(Dense(1))
    self.D.add(Activation('sigmoid'))
    self.D.summary()

    self.G = Sequential()
    dropout = 0.4
    depth = 64+64+64+64
    dim = 7
    # In: 100
    # Out: dim x dim x depth
    self.G.add(Dense(dim*dim*depth, input_dim=100))
    self.G.add(BatchNormalization(momentum=0.9))
    self.G.add(Activation('relu'))
    self.G.add(Reshape((dim, dim, depth)))
    self.G.add(Dropout(dropout))
    # In: dim x dim x depth
    # Out: 2*dim x 2*dim x depth/2
    self.G.add(UpSampling2D())
    self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
    self.G.add(BatchNormalization(momentum=0.9))
    self.G.add(Activation('relu'))
    self.G.add(UpSampling2D())
    self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
    self.G.add(BatchNormalization(momentum=0.9))
    self.G.add(Activation('relu'))
    self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
    self.G.add(BatchNormalization(momentum=0.9))
    self.G.add(Activation('relu'))
    # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
    self.G.add(Conv2DTranspose(1, 5, padding='same'))
    self.G.add(Activation('sigmoid'))
    self.G.summary()

    optimizer = RMSprop(lr=0.0008, clipvalue=1.0, decay=6e-8)
    self.DM = Sequential()
    self.DM.add(self.discriminator())
    self.DM.compile(loss='binary_crossentropy', optimizer=optimizer,\
    metrics=['accuracy'])

    optimizer = RMSprop(lr=0.0004, clipvalue=1.0, decay=3e-8)
    self.AM = Sequential()
    self.AM.add(self.generator())
    self.AM.add(self.discriminator())
    self.AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
    metrics=['accuracy'])

    self.img_rows

    images_train = self.x_train[np.random.randint(0, self.x_train.shape[0], size=batch_size), :, :, :]
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
    images_fake = self.generator.predict(noise)
    x = np.concatenate((images_train, images_fake))
    y = np.ones([2*batch_size, 1])
    y[batch_size:, :] = 0
    d_loss = self.discriminator.train_on_batch(x, y)
    y = np.ones([batch_size, 1])
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
    a_loss = self.adversarial.train_on_batch(noise, y)
gan = GAN(32)

AttributeError: 'GAN' object has no attribute 'img_rows'

TypeError: Discriminator() missing 1 required positional argument: 'self'